### Dependencies:

In [ ]:
import yaml
from datetime import datetime
import pandas as pd

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException

In [ ]:
edge:object = webdriver.Edge(executable_path="./utils/msedgedriver.exe")

### Functions:

In [ ]:
def resources(name:str) -> dict[str:str]:
    """Read xpath.yml file that contains xpath's to scrape"""

    with open("xpath.yml", "r") as f:
        content:dict[str:str] = yaml.safe_load(f)
        return content[name]

In [ ]:
def close_prompt(driver:object) -> None:
    """Close ad window"""

    button:str = "//a[@class='btn btn-subscription-light']"
    driver.find_element(by=By.XPATH, value=button).click()

In [ ]:
def choose_category(category:str, driver:object) -> None:
    """Choose ad category"""

    driver.find_element(by=By.XPATH, value=category).click()

In [ ]:
def click_element(element:object, path:str) -> None:
    """Click element on the current document"""

    button = element.find_element(by=By.XPATH, value=path)
    button.click()

In [ ]:
def extract_description(element:object, path:str) -> str:
    """Extract ad description on the current document"""

    description:str = element.find_element(by=By.XPATH, value=path)
    return description

In [ ]:
def extract_prices(element:object, path:str) -> list[str]:
    """Extract prices from list document"""

    try:
        prices:str = element.find_elements(by=By.XPATH, value=path)
        prices = [price.text for price in prices]
        return prices
    except Exception as e:
    prices.append("Web wasn't able to scrape prices")


In [ ]:
def extract_models(element:object, path:str) -> list[str]:
    """Extract models from list document"""

    try:
        models:str = element.find_elements(by=By.XPATH, value=path)
        models = [model.text for model in models]
        return models
    except Exception as e:
    models.append("Web wasn't able to scrape models")


In [ ]:
def persistence(dataframe:pd.Dataframe) -> None:
    """Generate .csv file from data scraped"""

    file_name = datetime.today().strftime("%B-%d")
    dataframe.to_csv(f"persistence/{file_name}.csv", index=False)

In [ ]:
def parse_prices(dataframe: pd.DataFrame, column:str) -> None:
    """
    Cast string prices to int prices.
    Works with $#,### or $#,###\n* pattern
    """

    price_rg:str = r"^\$(?P<prices>\d+,\d+)?"

    dataframe[column] = df[column] \
        .str.extract(price_rg) \
        .applymap(lambda price: int(price.replace(",","")))


## Extraction 🧲

### Sedan:

Variables pool.

In [ ]:
descriptions:list[str] = []
prices:list[str] = []
models:list[str] = []
# seconds
DELAY:int = 3

In [ ]:
sedan = resources("cars_sedan")

Get web & close initial prompt.

In [ ]:
edge.get(sedan["url"])
close_prompt(edge)

In [ ]:
elements = edge.find_elements(by=By.XPATH, value=sedan["ad"])

Extract descriptions.

In [ ]:
elements[0].click()
try:
    for element in range(len(elements)):
        dynamic_delay = WebDriverWait(edge, DELAY).until(EC.presence_of_element_located((By.XPATH, sedan["description"])))
        temp = extract_description(edge, sedan["description"])
        descriptions.append(temp.text)
        click_element(edge, sedan["next"])
except TimeoutException as TE:
    descriptions.append("Web wasn't able to scrape descriptions")
except NoSuchElementException as NSE:
    click_element(edge, sedan["exit"])

Extract prices & models.

In [ ]:
prices = extract_prices(edge, sedan["price"])
models = extract_models(edge, sedan["model"])
models.remove("Blue Book Encuentra24")

## Procesing ⚙

In [ ]:
# working with persistence data
df = pd.read_csv("./persistence/February-10.csv")

Remove invasive ad from list document.

### Price column:

Chance price dtype from str -> int

In [ ]:
parce_prices(df)

In [ ]:
df.info()